In [24]:
# Import libraries

import pandas as pd
import json
from sentence_transformers import SentenceTransformer
import minsearch2
from tqdm.autonotebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Read JSON document with generated ids
with open('..\data\data-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

# Load generated ground truth data
df_questions = pd.read_csv('..\data\ground-truth-retrieval.csv')
ground_truth = df_questions.to_dict(orient='records')

In [4]:
doc_idx = {d['id']: d for d in documents}

In [5]:
# Define model to use for embeddings
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [40]:
# Define text fields with combinations
text_fields = ['question', 'answer', 'source', 'focus_area', 'question_answer', 'question_answer_focus', 'question_focus', 'answer_focus']

# Create combined fields in the documents
for doc in documents:
    doc['question_answer'] = doc['question'] + " " + doc['answer']
    doc['question_answer_focus'] = doc['question'] + " " + doc['answer'] + " " + doc['focus_area']
    doc['question_focus'] = doc['question'] + " " + doc['focus_area']
    doc['answer_focus'] = doc['answer'] + " " + doc['focus_area']

  0%|          | 0/292 [00:00<?, ?it/s]

In [42]:
# Create the index using the modified minsearch function and fit documents
index = minsearch2.Index(text_fields=text_fields, keyword_fields=['id'])
encoded_vectors = {field: [model.encode(doc[field]) for doc in documents] for field in text_fields} #generate embeddings

index.fit(documents, encoded_vectors)

In [43]:
# Search function
def minsearch_search(field, query_vector):
    boost = {field: 1}
    query = {field: query_vector}
    results = index.search(
        query_vectors=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )
    return results

In [9]:
# Hit rate
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

# mrr
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank]:  # If relevant document found
                total_score += 1 / (rank + 1)
                break

    return total_score / len(relevance_total)

In [10]:
# Evaluate the performance
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [44]:
def question_vector_knn(q):
    question = q['question']
    v_q = model.encode(question)
    return minsearch_search('question', v_q)

# Evaluation
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/1460 [00:00<?, ?it/s]

{'hit_rate': 0.9732876712328767, 'mrr': 0.9361934659708627}

Combine all the KNN functions

In [51]:
def vector_knn(q, field):
    question = q['question']
    v_q = model.encode(question)
    return minsearch_search(field, v_q)

# Question only
def question_vector_knn(q):
    return vector_knn(q, 'question')
    
# Answer only
def answer_vector_knn(q):
    return vector_knn(q, 'answer')

# Question and Answer
def question_answer_vector_knn(q):
    return vector_knn(q, 'question_answer')

# Question, Answer and Focus area
def question_answer_focus_vector_knn(q):
    return vector_knn(q, 'question_answer_focus')

# Question and Focus area
def question_focus_vector_knn(q):
    return vector_knn(q, 'question_focus')

# Answer and Focus area
def answer_focus_vector_knn(q):
    return vector_knn(q, 'answer_focus')

#### **Vector Evaluation Results**

In [ ]:
def run_evaluations(ground_truth):
    results = {}

    results['question_vector'] = evaluate(ground_truth, question_vector_knn)
    results['answer_vector'] = evaluate(ground_truth, answer_vector_knn)
    results['question_answer_vector'] = evaluate(ground_truth, question_answer_vector_knn)
    results['question_answer_focus_vector'] = evaluate(ground_truth, question_answer_focus_vector_knn)
    results['question_focus_vector'] = evaluate(ground_truth, question_focus_vector_knn)
    results['answer_focus_vector'] = evaluate(ground_truth, answer_focus_vector_knn)
    
    return results

In [55]:
def print_evaluation_results(results, sort_by='hit_rate'):
    sorted_results = sorted(
        results.items(), 
        key=lambda item: item[1][sort_by], 
        reverse=True
    )

    for knn_type, metrics in sorted_results:
        print(f"{knn_type}: Hit Rate: {metrics['hit_rate']:.4f}, MRR: {metrics['mrr']:.4f}")


results = run_evaluations(ground_truth)
# Print results sorted by 'hit_rate'
print("Evaluation Results Sorted by Hit Rate:")
print_evaluation_results(results, sort_by='hit_rate')

  0%|          | 0/1460 [00:00<?, ?it/s]

  0%|          | 0/1460 [00:00<?, ?it/s]

  0%|          | 0/1460 [00:00<?, ?it/s]

  0%|          | 0/1460 [00:00<?, ?it/s]

  0%|          | 0/1460 [00:00<?, ?it/s]

  0%|          | 0/1460 [00:00<?, ?it/s]

Evaluation Results Sorted by Hit Rate:
question_answer_vector: Hit Rate: 0.9925, MRR: 0.9561
question_answer_focus_vector: Hit Rate: 0.9911, MRR: 0.9518
answer_focus_vector: Hit Rate: 0.9904, MRR: 0.9353
answer_vector: Hit Rate: 0.9788, MRR: 0.9050
question_vector: Hit Rate: 0.9733, MRR: 0.9362
question_focus_vector: Hit Rate: 0.9705, MRR: 0.9262


Combination of the question and answer vectors produce the highest results of both the hit rate and mrr.

Vector search has better results than text search.